# Implanatação de algumas Bibliotacas

In [ ]:
import csv
import numpy as np
import pandas as pd
import sys
from tkinter import Tk
from tkinter import filedialog
from datetime import date, datetime, timedelta
#import os
#from ciso8601 import parse_datetime

# Criando variaveis para os arquivos permitidos

In [ ]:
p_cmpid5  = pd.read_excel('PQ380.xlsx')      ### leitura do arquivo com os sinais dos cpmids do quadro 380
param5    = pd.read_excel('LQ380.xlsx')      ### leitura do arquivo com o layout atualizado do quadro 380

In [ ]:
control  = 0                                              ### controle de backup    
p_cmpid5['cmpid'] = p_cmpid5.cmpid.astype(str)            ### transforma o campo cmpid - QE 380 - do arq auxiliar para string

In [ ]:
## FUNÇÃO PARA SELEÇÃO DO ARQUIVO

class qe_s:

    def __init__(self):
        self.df = {nome[s]: [] for s in range(nome.shape[0])}
  
    def make_df(self):
        lines = []
        root = Tk()
        root.attributes("-topmost", True)
        root.withdraw()
        for file in filedialog.askopenfilenames(filetypes=[("Arquivos de Text", "*.txt")]):
            for line in open(file, encoding="utf-8"):
                line = line.replace(",",".").strip()
                for x in range(nome.shape[0]):
                    self.df[nome[x]].append(line[i[x]:j[x]])
                    
        return pd.DataFrame(self.df)

In [ ]:
nome     = param5.Campo                      ### guarda os nomes das colunas existentes no quadro 380
i        = param5.Ind_inf                    ### guarda a posição no arquivo que inicia um determinado campo
j        = param5.Ind_sup                    ### guarda a posição no arquivo que termina um determinado campo

quadro = qe_s()       
df_main = quadro.make_df()
df_main

## SELECINE O(S) ARQUIVO(S) DESEJADO(S)

In [ ]:
###################################
###Importacao do Quadro380.xlsx ###
####que serão usados para fins#####
######### de comparação ###########
###################################

#Pasta CMPID
arq_cmpid = pd.read_excel('Quadro380.xlsx','CMPID',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta RAMCODIGO
arq_ramcod = pd.read_excel('Quadro380.xlsx','RAMCODIGO',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta ENTCODIGO
arq_entcod = pd.read_excel('Quadro380.xlsx','Cod_SUSEP',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta MRFMESANO
arq_mrfmesano = pd.read_excel('Quadro380.xlsx','MRFMESANO',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta rel_CMPID_TPMOID
arq_relacao = pd.read_excel('Quadro380.xlsx','rel_CMPID_TPMOID',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta TPMOID
arq_tpmoid = pd.read_excel('Quadro380.xlsx','TPMOID',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta Bib_DefCamposEstatísticos
arq_defcamesta = pd.read_excel('Quadro380.xlsx','Bib_DefCamposEstatísticos',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta TPMOPREV
arq_defcamesta = pd.read_excel('Quadro380.xlsx','TPMOPREV',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

In [ ]:
# Criando uma lista para guardar todas as criticas impeditivas
criticas_impeditivas = []
criticas = []

# Criticas (Validacoes)

In [ ]:
#7396.3 Verifica se o campo sequencial EBRSEQ é uma sequência válida, que se inicia em 0000001
array3 = df_main['EBRSEQ']
x = list(array3)                                     ##Dados do arquivo
arrayteste = x[:]
arrayteste.sort()                                    ##Dados sequenciados
if (arrayteste == x):                                ##Comparando
    print('Tudo certo, esta de forma sequencial.')
else: 
    print('ERRO!!!, Não prossiga.')
    critica7396_3 = str('Critica 7396.3, Não temos uma sequencia valida na coluna EBRSEQ')
    criticas_impeditivas.append(critica7396_3)

In [ ]:
#7396.2 Verifica o tamanho padrão da linha (146 caracteres)
tamanho = 0
for i in range(0,df_main.shape[0]):                    #Passando por cada linha
    for iten in df_main.columns:                       #Passando por cada coluna
        tamanho = tamanho + len(df_main[iten][i])      #Contando Caracteres por linha
    if tamanho != 146:
        print('Erro na linha ',i,' o número de caracteres é de: ',tamanho)
        critica7396_2 = ("Critica 7396.2, Erro na linha ",i," o número de caracteres é de: ",tamanho)
        criticas_impeditivas.append(critica7396_2)
    tamanho = 0

In [ ]:
#7396.4 Verifica se o campo ENTCODIGO corresponde à sociedade que está enviando o FIP/SUSEP
entcod = set(arq_entcod['Cod_SUSEP'].astype('str'))                   #Escolhendo a coluna do arquivo modelo
dadosentcod = set(df_main['ENTCODIGO'])                               #Escolhendo a coluna dos dados
for itemtentcod in dadosentcod: 
    if itemtentcod not in entcod:
        print('ERRO!! ENTCODIGO não corresponde a uma operação valida', itemtentcod)
        critica7396_4 = ('Critica 7396.4, ENTCODIGO não corresponde a uma operação valida', itemtentcod)
        criticas_impeditivas.append(critica7396_4)
    else:
        print('ENTCODIGO corresponde a um tipo de operação válida.', itemtentcod)
        continue

In [ ]:
#7396.5 Verifica se o campo MRFMESANO corresponde, respectivamente, ao ano, mês e último dia do mês de referência do FIP/SUSEP
df_main.loc[:,'DTMRFMESANO'] = df_main['MRFMESANO'].astype('datetime64')     #Criando uma nova coluna no formato Date
mrfmesano = set(arq_mrfmesano['DATAS'].astype('datetime64'))                 #Escolhendo a coluna do arquivo modelo
dadosmrfmesano = set(df_main['DTMRFMESANO'])                                 #Escolhendo a coluna dos dados
for itemmrfmesano in dadosmrfmesano: 
    if itemmrfmesano not in mrfmesano:
        print('ERRO!! MRFMESANO não corresponde a uma operação valida', itemmrfmesano)
        critica7396_5 = ('Critica 7396.5, MRFMESANO não corresponde a uma Data valida', itemmrfmesano)
        criticas_impeditivas.append(critica7396_5)
    else:
        print('MRFMESANO corresponde a uma Data válida.', itemmrfmesano)
        continue

In [ ]:
#7396.6 Verifica se o campo QUAID corresponde ao quadro 380 
array6 = set(df_main['QUAID'])           ##Verificando todos os Quadros que estamos tratando no documento.
for itemarray6 in array6:
    if itemarray6 == '380':
        print('Este se trata do Quadro:',itemarray6)
    else:
        print('ERRO!! Este se trata do Quadro:',itemarray6)
        critica7396_6 = ('Critica 7396.6, Este esta se tratando do(s) Quadro(s):',itemarray6)
        criticas_impeditivas.append(critica7396_6)

In [ ]:
#7396.7 Verifica se o campo CMPID corresponde a um tipo de operação válida (conforme tabela “Bib_DefCampos”)
#defcamesta = set(arq_defcamesta['CMPID'].astype('str'))             #Escolhendo a coluna do arquivo modelo
cmpid = set(arq_cmpid['CMPID'].astype('str'))
dadoscmpid = set(df_main['CMPID'])                                 #Escolhendo a coluna dos dados
for itemcmpid in dadoscmpid: 
    if itemcmpid not in cmpid:
        print('ERRO!! O campo CMPID não corresponde a um tipo de operação válida (conforme tabela Bib_DefCamposEstatísticos).', itemcmpid)
        critica7396_7 = ('Critica 7396.7, O campo CMPID não corresponde a um tipo de operação válida (conforme tabela Bib_DefCamposEstatísticos).', itemcmpid)
        criticas.append(critica7396_7)
    else:
        print('O campo CMPID corresponde a um tipo de operação válida (conforme tabela Bib_DefCamposEstatísticos).', itemcmpid)
        continue

In [ ]:
#7396.8 Verifica se o PLNCODIGO pertence ao Cadastro de Planos da entidade 

In [ ]:
#7396.9 Verifica se o CPF do participante (EBRCPFPART) e do beneficiário (EBRCPFBENF) são inteiros e válidos, exceto para preenchimento com zeros

#Criação da função
import re

def isCpfValid(cpf):
    """ If cpf in the Brazilian format is valid, it returns True, otherwise, it returns False. """

    # Check if type is str
    if not isinstance(cpf,str):
        return False

    # Remove some unwanted characters
    cpf = re.sub("[^0-9]",'',cpf)
    
    # Verify if CPF number is equal
    if cpf=='00000000000' or cpf=='11111111111' or cpf=='22222222222' or cpf=='33333333333' or cpf=='44444444444' or cpf=='55555555555' or cpf=='66666666666' or cpf=='77777777777' or cpf=='88888888888' or cpf=='99999999999':
        return False

    # Checks if string has 11 characters
    if len(cpf) != 11:
        return False

    sum = 0
    weight = 10

    """ Calculating the first cpf check digit. """
    for n in range(9):
        sum = sum + int(cpf[n]) * weight

        # Decrement weight
        weight = weight - 1

    verifyingDigit = 11 -  sum % 11

    if verifyingDigit > 9 :
        firstVerifyingDigit = 0
    else:
        firstVerifyingDigit = verifyingDigit

    """ Calculating the second check digit of cpf. """
    sum = 0
    weight = 11
    for n in range(10):
        sum = sum + int(cpf[n]) * weight

        # Decrement weight
        weight = weight - 1

    verifyingDigit = 11 -  sum % 11

    if verifyingDigit > 9 :
        secondVerifyingDigit = 0
    else:
        secondVerifyingDigit = verifyingDigit

    if cpf[-2:] == "%s%s" % (firstVerifyingDigit,secondVerifyingDigit):
        return True
    return False

In [ ]:
# Realizando Check no Campo EBRCPFPART
cpfparticipante = set(df_main['EBRCPFPART'])

for itemcpfp in cpfparticipante:
    verificacao1 = isCpfValid(itemcpfp)
    if verificacao1 == False:
        print("Erro!! Critica 7395.10 o CPF ", itemcpfp, " não é um CPF valido.")
        critica7396_10 = ("Critica 7396.10","o CPF ", itemcpfp, " não é um CPF valido.")
        criticas_impeditivas.append(critica7396_10)

print('Demais CPF do participante nos campos (EBRCPFPART) são validos') 

In [ ]:
# Realizando Check no Campo EBRCPFBENF
cpfbeneficiario = set(df_main['EBRCPFBENF'])

for itemcpfb in cpfbeneficiario:
    verificacao2 = isCpfValid(itemcpfb)
    if verificacao2 == False:
        print("Erro!! Critica 7396.10 o CPF ", itemcpfb, " não é um CPF valido.")
        critica7396_11 = ("Critica 7396.10","o CPF ", itemcpfb, " não é um CPF valido.")
        criticas_impeditivas.append(critica7396_10)

print('Demais CPF do participante nos campos (EBRCPFBENF) são validos') 

In [ ]:
#7396.10 Verifica se os campos EBRDATAINICIO, EBRDATAFIM, EBRDATAOCORR e EBRDATAREG correspondem a uma data válida e se estão compreendidos entre os anos de 1901 e 2099 

#####################################################
##Readicionando as colunas de Data de str para date##
#####################################################

df_main.loc[:,'DTEBRDATAINICIO'] = df_main['EBRDATAINICIO'].astype('datetime64')
df_main.loc[:,'DTEBRDATAFIM'] = df_main['EBRDATAFIM'].astype('datetime64')
df_main.loc[:,'DTEBRDATAOCORR'] = df_main['EBRDATAOCORR'].astype('datetime64')
df_main.loc[:,'DTEBRDATAREG'] = df_main['EBRDATAREG'].astype('datetime64')

#Adicionado colunas EBRDATAINICIO, EBRDATAFIM, EBRDATAOCORR e EBRDATAREG no tipo Date

df_dates_comparacao = pd.DataFrame(df_main['DTMRFMESANO'] - timedelta(days = 43800))
df_dates_comparacao.loc[:,'DTMRFMESANO_max'] = df_main['DTMRFMESANO'] + timedelta(days = 29200)

In [ ]:
comparação_01 = (df_main['DTEBRDATAINICIO'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTEBRDATAINICIO'] > df_dates_comparacao['DTMRFMESANO_max'])
dfc01 = df_main.loc[comparação_01]
if len(dfc01) > 0:
    dfc01.loc[:,'ID da Critica'] = ('CRITICA 7396.10')
dfc01

In [ ]:
comparação_02 = (df_main['DTEBRDATAFIM'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTEBRDATAFIM'] > df_dates_comparacao['DTMRFMESANO_max'])
dfc02 = df_main.loc[comparação_02]
if len(dfc02) > 0:
    dfc02.loc[:,'ID da Critica'] = ('CRITICA 7396.10')
dfc02

In [ ]:
comparação_03 = (df_main['DTEBRDATAOCORR'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTEBRDATAOCORR'] > df_dates_comparacao['DTMRFMESANO_max'])
dfc03 = df_main.loc[comparação_03]
if len(dfc03) > 0:
    dfc03.loc[:,'ID da Critica'] = ('CRITICA 7396.10')
dfc03

In [ ]:
comparação_04 = (df_main['DTEBRDATAREG'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTEBRDATAREG'] > df_dates_comparacao['DTMRFMESANO_max'])
dfc04 = df_main.loc[comparação_04]
if len(dfc04) > 0:
    dfc04.loc[:,'ID da Critica'] = ('CRITICA 7396.10')
dfc04

In [ ]:
#7396.11  Verifica se o campo EBRVALORMOV é float 

###########################################
##Readicionando as colunas str para float##
###########################################

df_main.loc[:,'fEBRVALORMOV'] = df_main['EBRVALORMOV'].astype('float64')

#Adicionado coluna EBRVALORMOV no tipo Float

In [ ]:
criticas_impeditivas  #Lista das criticas impeditivas

In [ ]:
df_criticas_i = pd.DataFrame(criticas_impeditivas)     #criacao do DataFrame das criticas impeditivas
df_criticas_i = df_criticas_i.rename(columns={0:'Descrição'})
df_criticas_i

In [ ]:
df_criticas_i2 = pd.concat([dfc01,dfc02,dfc03,dfc04])
df_criticas_i2

In [ ]:
if criticas_impeditivas == []:                             # Verificando se a lista de criticas esta vazia
    print('Nenhuma Critica Impeditiva encontrada no Quadro 380')
else:
    print('Foi encontrada uma ou mais Criticas Impeditivas no Quadro 380')
    df_criticas_i.to_csv('Criticas_Impeditivas_380.csv')   # Gerando csv das criticas impeditivas

In [ ]:
if len(df_criticas_i2) == 0:                             # Verificando se a lista de criticas esta vazia
    print('Nenhuma Critica Impeditiva encontrada no Quadro 380')
else:
    print('Foi encontrada uma ou mais Criticas Impeditivas no Quadro 380')
    df_criticas_i2.to_csv('Criticas_Impeditivas_380_2.csv')   # Gerando csv das criticas impeditivas

# Criticas (Validacoes) Não Impeditivas

In [ ]:
#7396.12 Verifica se o campo EBRCODCESS corresponde a um código de sociedade válido 
dadosebicodcess = set(df_main['EBRCODCESS'])                               #Escolhendo a coluna dos dados
for itemebicodcess in dadosebicodcess: 
    if itemebicodcess not in entcod:
        print('ERRO!! EBICODCESS não corresponde a um código de sociedade válido', itemebicodcess)
        critica7396_12 = ('Critica 7396.12, EBICODCESS não corresponde a um código de sociedade válido', itemebicodcess)
        criticas.append(critica7396_12)
    else:
        print('EBICODCESS corresponde a um código de sociedade válido.', itemebicodcess)
        continue

In [ ]:
# Criando DataFrame copia com Campos CMPID, EBRCODCESS, ENTCODIGO no formato int
df_mains = df_main.copy()
df_mains['CMPID'] = df_mains['CMPID'].astype('int64')
df_mains['EBICODCESS'] = df_mains['EBRCODCESS'].astype('int64')
df_mains['ENTCODIGO'] = df_mains['ENTCODIGO'].astype('int64')

In [ ]:
#7396.13 Valida a correspondência entre os campos CMPID e EBRCODCESS 
relacao13_53 = df_mains[(df_mains['CMPID'] == 1053) & (df_mains['EBICODCESS'].values != df_mains['ENTCODIGO'].values)]
relacao13_58 = df_mains[(df_mains['CMPID'] == 1058) & (df_mains['EBICODCESS'].values != df_mains['ENTCODIGO'].values)]
relacao13_63 = df_mains[(df_mains['CMPID'] == 1063) & (df_mains['EBICODCESS'].values != df_mains['ENTCODIGO'].values)]
relacao13_64 = df_mains[(df_mains['CMPID'] == 1064) & (df_mains['EBICODCESS'].values != df_mains['ENTCODIGO'].values)]

In [ ]:
i = 23   #Definindo numero de colunas

# Adicionando criticas com CMPIDs 1053, 1058, 1063, 1064 na lista 'criticas'
if relacao13_53.shape != (0,i): 
    critica7395_13_53 = ('Critica 7396.13, O CMPID é 1053 e o EBICODCESS é diferente do ENTCODIGO') 
    criticas.append(critica7395_17_38)
elif relacao13_58.shape != (0,i): 
    critica7392_13_58 = ('Critica 7396.13, O CMPID é 1058 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_17_39)
elif relacao13_63.shape != (0,i): 
    critica7392_13_63 = ('Critica 7396.13, O CMPID é 1063 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_17_42)
elif relacao13_64.shape != (0,i): 
    critica7392_13_64 = ('Critica 7396.13, O CMPID é 1064 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_17_44)
else: print('CMPIDs 1053, 1058, 1063, 1064. Valida campo EBICODCESS')

In [ ]:
#7396.13 Valida a correspondência entre os campos CMPID e EBRCODCESS 
relacao13_54 = df_mains[(df_mains['CMPID'] == 1054) & (df_mains['EBICODCESS'].values <= 1) & (df_mains['EBICODCESS'].values >= 9999)] 
relacao13_55 = df_mains[(df_mains['CMPID'] == 1055) & (df_mains['EBICODCESS'].values <= 1) & (df_mains['EBICODCESS'].values >= 9999)] 
relacao13_59 = df_mains[(df_mains['CMPID'] == 1059) & (df_mains['EBICODCESS'].values <= 1) & (df_mains['EBICODCESS'].values >= 9999)] 
relacao13_60 = df_mains[(df_mains['CMPID'] == 1060) & (df_mains['EBICODCESS'].values <= 1) & (df_mains['EBICODCESS'].values >= 9999)]

In [ ]:
# Adicionando criticas com CMPIDs 1054, 1055, 1059, 1060 na lista 'criticas'
if relacao13_54.shape != (0,i): 
    critica7395_13_54 = ('Critica 7396.13, O CMPID é 1054 e o EBICODCESS é diferente do ENTCODIGO') 
    criticas.append(critica7395_13_54)
elif relacao13_55.shape != (0,i): 
    critica7392_13_55 = ('Critica 7396.13, O CMPID é 1055 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_13_55)
elif relacao13_59.shape != (0,i): 
    critica7392_13_59 = ('Critica 7396.13, O CMPID é 1059 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_13_59)
elif relacao13_60.shape != (0,i): 
    critica7392_13_60 = ('Critica 7396.13, O CMPID é 1060 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_13_60)
else: print('CMPIDs 1054, 1055, 1059, 1060. Valida campo EBICODCESS')

In [ ]:
#7396.13 Valida a correspondência entre os campos CMPID e EBRCODCESS 
relacao13_56 = df_mains[(df_mains['CMPID'] == 1056) & (df_mains['EBICODCESS'].values <= 30000) & (df_mains['EBICODCESS'].values >= 59999)] 
relacao13_57 = df_mains[(df_mains['CMPID'] == 1057) & (df_mains['EBICODCESS'].values <= 30000) & (df_mains['EBICODCESS'].values >= 59999)] 
relacao13_61 = df_mains[(df_mains['CMPID'] == 1061) & (df_mains['EBICODCESS'].values <= 30000) & (df_mains['EBICODCESS'].values >= 59999)] 
relacao13_62 = df_mains[(df_mains['CMPID'] == 1062) & (df_mains['EBICODCESS'].values <= 30000) & (df_mains['EBICODCESS'].values >= 59999)]

In [ ]:
# Adicionando criticas com CMPIDs 1056, 1057, 1061, 1062 na lista 'criticas'
if relacao13_56.shape != (0,i): 
    critica7395_13_56 = ('Critica 7396.13, O CMPID é 1056 e o EBICODCESS é diferente do ENTCODIGO') 
    criticas.append(critica7395_13_56)
elif relacao13_57.shape != (0,i): 
    critica7392_13_57 = ('Critica 7396.13, O CMPID é 1057 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_13_57)
elif relacao13_61.shape != (0,i): 
    critica7392_13_61 = ('Critica 7396.13, O CMPID é 1061 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_13_61)
elif relacao13_62.shape != (0,i): 
    critica7392_13_62 = ('Critica 7396.13, O CMPID é 1062 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_13_62)
else: print('CMPIDs 1056, 1057, 1061, 1062. Valida campo EBICODCESS')

In [ ]:
#7396.15 Verifica se o CPF do participante (EBRCPFPART) e do beneficiário (EBRCPFBENF) são diferentes de zero 
for itemcpfp in cpfparticipante:
    if itemcpfp == '00000000000':
        print("Erro!! Critica 7395.20 o CPF ", itemcpfp, " não é um CPF valido.")
        critica7396_15 = ("Critica 7396.15","o CPF ", itemcpfp, " não é um CPF valido.")
        criticas.append(critica7396_15)

print('CPFs dos participantes nos campos (EBICPFPART) são validos')

In [ ]:
for itemcpfb in cpfbeneficiario:
    if itemcpfb == '00000000000':
        print("Erro!! Critica 7395.20 o CPF ", itemcpfb, " não é um CPF valido.")
        critica7396_15 = ("Critica 7396.15","o CPF ", itemcpfb, " não é um CPF valido.")
        criticas.append(critica7396_15)

print('CPFs dos participantes nos campos (EBICPFBENF) são validos')

In [ ]:
#7396.16 Verifica se a data do campo EBRDATAFIM é posterior à data do campo EBRDATAINICIO
mask16 = (df_main['DTEBRDATAFIM']) < (df_main['DTEBRDATAINICIO'])
df16 = df_main.loc[mask16]
if len(df16) > 0:
    df16.loc[:,'ID da Critica'] = ('CRITICA 7396.16')
df16

In [ ]:
#7396.17 Verifica se a data do campo EBRDATAINICIO é igual ou anterior à data do campo EBRDATAOCORR
mask17 = (df_main['DTEBRDATAINICIO']) > (df_main['DTEBRDATAOCORR'])
df17 = df_main.loc[mask17]
if len(df17) > 0:
    df17.loc[:,'ID da Critica'] = ('CRITICA 7396.17')
df17

In [ ]:
#7396.18 Verifica se a data do campo EBRDATAOCORR é igual ou anterior à data do campo EBRDATAFIM
mask18 = (df_main['DTEBRDATAFIM']) < (df_main['DTEBRDATAOCORR'])
df18 = df_main.loc[mask18]
if len(df18) > 0:
    df18.loc[:,'ID da Critica'] = ('CRITICA 7396.18')
df18

In [ ]:
#7396.19 Verifica se a data do campo EBRDATAOCORR é igual ou anterior à data do campo EBRDATAREG
mask19 = (df_main['DTEBRDATAREG']) < (df_main['DTEBRDATAOCORR'])
df19 = df_main.loc[mask19]
if len(df19) > 0:
    df19.loc[:,'ID da Critica'] = ('CRITICA 7396.19')
df19

In [ ]:
#7396.20 Verifica se a data do campo EBRDATAREG é igual ou anterior à data do campo MRFMESANO
mask20 = (df_main['DTEBRDATAREG']) > (df_main['DTMRFMESANO'])
df20 = df_main.loc[mask20]
if len(df20) > 0:
    df20.loc[:,'ID da Critica'] = ('CRITICA 7396.20')
df20

In [ ]:
#7396.22 Verifica se o ano da data dos campos EBRDATAINICIO, EBRDATAFIM, EBRDATAOCORR e EBRDATAREG está entre os limites de 
#trinta anos para mais ou para menos do ano da data do campo MRFMESANO quando o tipo de operação for 'direto – administrativo' 
#(CMPID: 1053), 'cosseguro aceito – administrativo’ (CMPID: 1054),'cosseguro cedido - administrativo' (CMPID: 1055), 
#'recuperação de sinistros não pagos - administrativo' (CMPID: 1056), 'recuperação de sinistros já pagos - administrativo' (CMPID: 1057)  
df_dates_comparacao.loc[:,'DTMRFMESANO+30anos'] = df_main['DTMRFMESANO'] + timedelta(days = 10950)
df_dates_comparacao.loc[:,'DTMRFMESANO-30anos'] = df_main['DTMRFMESANO'] - timedelta(days = 10950)

mask22_cmpid = (df_main['CMPID'] == '1053')|(df_main['CMPID'] == '1054')|(df_main['CMPID'] == '1055')|(df_main['CMPID'] == '1056')|(df_main['CMPID'] == '1057')

In [ ]:
mask22_INI1 = (df_main['DTEBRDATAINICIO'] < df_dates_comparacao['DTMRFMESANO-30anos']) | (df_main['DTEBRDATAINICIO'] > df_dates_comparacao['DTMRFMESANO+30anos'])
mask22_INI2 = (mask22_cmpid) & (mask22_INI1)
df22_ini = df_main.loc[mask22_INI2]
if len(df22_ini) > 0:
    df22_ini.loc[:,'ID da Critica'] = ('CRITICA 7396.22')
df22_ini

In [ ]:
mask22_fim1 = (df_main['DTEBRDATAFIM'] < df_dates_comparacao['DTMRFMESANO-30anos']) | (df_main['DTEBRDATAFIM'] > df_dates_comparacao['DTMRFMESANO+30anos'])
mask22_fim2 = (mask22_cmpid) & (mask22_fim1)
df22_fim = df_main.loc[mask22_fim2]
if len(df22_fim) > 0:
    df22_fim.loc[:,'ID da Critica'] = ('CRITICA 7396.22')
df22_fim

In [ ]:
mask22_oco1 = (df_main['DTEBRDATAOCORR'] < df_dates_comparacao['DTMRFMESANO-30anos']) | (df_main['DTEBRDATAOCORR'] > df_dates_comparacao['DTMRFMESANO+30anos'])
mask22_oco2 = (mask22_cmpid) & (mask22_oco1)
df22_oco = df_main.loc[mask22_oco2]
if len(df22_oco) > 0:
    df22_oco.loc[:,'ID da Critica'] = ('CRITICA 7396.22')
df22_oco

In [ ]:
mask22_reg1 = (df_main['DTEBRDATAREG'] < df_dates_comparacao['DTMRFMESANO-30anos']) | (df_main['DTEBRDATAREG'] > df_dates_comparacao['DTMRFMESANO+30anos'])
mask22_reg2 = (mask22_cmpid) & (mask22_reg1)
df22_reg = df_main.loc[mask22_reg2]
if len(df22_reg) > 0:
    df22_reg.loc[:,'ID da Critica'] = ('CRITICA 7396.22')
df22_reg

In [ ]:
#7396.23 Para cada PLNCODIGO, verifica se o resultado da soma do EBRVALORMOV para os movimentos 'direto – administrativo' (CMPID: 1053) e 'cosseguro aceito – administrativo’ (CMPID: 1054) subtraída da soma do EBRVALORMOV dos movimentos de 'cosseguro cedido - administrativo' (CMPID: 1055) é igual à ‘PSL/Administrativa/Avisada’ dos Quadros por plano 110 (CMPID: 12914), 111(CMPID: 12953), 112 (CMPID: 12992) e 113 (CMPID: 13031) 

In [ ]:
#7396.24 Para cada PLNCODIGO, verifica se o resultado da soma do EBRVALORMOV para os movimentos 'direto – judicial’ (CMPID: 1058), ‘Revisão Judicial de Benefícios Concedidos (PMBC)’ (CMPID: 1064) e 'cosseguro aceito – judicial’ (CMPID: 1059) subtraída da soma do EBRVALORMOV dos movimentos de 'cosseguro cedido - judicial' (CMPID: 1060) é igual à ‘PSL/Judicial/Avisada’ dos Quadros por plano 110 (CMPID: 12918), 111(CMPID: 12957), 112 (CMPID: 12996) e 113 (CMPID: 13035) 

In [ ]:
#7396.25 Soma dos campos EBRVALORMOV dos registros cujo CMPID seja 1063 (Depósito Judicial Redutor da Necessidade de Cobertura) deve ser igual ao valor informado no campo Depósitos Judiciais Redutores – Previdência/Dotais/VI/VGBL/Universal/EFPC (CMPID 12594) do quadro 16.

In [ ]:
#7396.26 Verifica se o valor do campo EBRNUMSIN é maior do que zero 
mask26 = (df_main['EBRNUMSIN'] == '00000000000000000000')
df26 = df_main.loc[mask26]
if len(df26) > 0:
    df26.loc[:,'ID da Critica'] = ('CRITICA 7396.26')
df26

In [ ]:
#7396.27 Verifica se o valor do campo EBRVALORMOV é maior do que zero 
mask27 = (df_main['EBRNUMSIN'] == 0.0)
df27 = df_main.loc[mask27]
if len(df27) > 0:
    df27.loc[:,'ID da Critica'] = ('CRITICA 7396.27')
df27

In [ ]:
################################################
# Adicionando Nome do Criador do output e data #
################################################

df_main.loc[:,'Criador'] = input("Insira seu nome: ") ### insere a coluna com o nome do criador da tabela
data_presente = datetime.now()
df_main.loc[:,'Data do output'] = data_presente ### insere a coluna com a data de criacao da tabela

In [ ]:
df_main

In [ ]:
criticas

In [ ]:
df_criticas = pd.DataFrame(criticas)
df_criticas = df_criticas.rename(columns={0:'Descrição'})
df_criticas

In [ ]:
df_criticas_2 = pd.concat([df16,df17,df18,df19,df20,df22_ini,df22_fim,df22_oco,df22_reg,df26,df27])
df_criticas_2

In [ ]:
if criticas == []:                   # Verificando se a lista de criticas esta vazia
    print('Nenhuma Critica NÃO Impeditiva encontrada no Quadro 380')
else:
    print('Foi encontrada uma ou mais Criticas NÃO Impeditivas no Quadro 380')
    df_criticas.to_csv('Criticas_nao_impeditivas380.csv')     # Gerando csv das criticas impeditivas

In [ ]:
if len(df_criticas_2) == 0:                             # Verificando se o DataFrame de criticas esta vazia
    print('Nenhuma Critica NÃO Impeditiva encontrada no Quadro 380')
else:
    print('Foi encontrada uma ou mais Criticas NÃO Impeditivas no Quadro 380')
    df_criticas_2.to_csv('Criticas_nao_impeditivas380_2.csv')         # Gerando csv das criticas impeditivas

In [ ]:
df_main.to_csv('380_Trabalho.csv')